In [5]:
import keras
from keras.layers import Input, Embedding, LSTM, Dense, Dropout
from keras.models import Model

In [6]:
img_maxlen = 100
img_input = Input(shape=(img_maxlen,))
x1 = Dense(1000, activation='relu')(img_input)
x1 = Dropout(0.5)(x1)
x1 = Dense(600, activation='relu')(x1)
img_output = Dense(10)(x1)

W1031 16:20:22.984114 140611263473472 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W1031 16:20:22.994876 140611263473472 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [7]:
text_maxlen = 100
text_input = Input(shape=(text_maxlen,),dtype='int32')
x2 = Embedding(output_dim=512, input_dim=10000, input_length=text_maxlen)(text_input)
x2 = LSTM(32)(x2)
x2 = Dropout(0.5)(x2)
text_output = Dense(10)(x2)

In [9]:
#用BERT来做文本处理
from keras_bert import load_trained_model_from_checkpoint, Tokenizer
from keras.layers import *

config_path = '/openbayes/input/input0/BERT/cased_L-12_H-768_A-12/bert_config.json'
checkpoint_path = '/openbayes/input/input0/BERT/cased_L-12_H-768_A-12/bert_model.ckpt'
dict_path = '/openbayes/input/input0/BERT/cased_L-12_H-768_A-12/vocab.txt'

bert_model = load_trained_model_from_checkpoint(config_path, checkpoint_path, seq_len=None)

for l in bert_model.layers:
    l.trainable = True

x1_in = Input(shape=(None,))
x2_in = Input(shape=(None,))

x = bert_model([x1_in, x2_in])
x = Lambda(lambda x: x[:, 0])(x) # 取出[CLS]对应的向量用来做分类

x = keras.layers.concatenate([img_output, x])

p = Dense(3, activation='sigmoid')(x)

#重新定义一个具有BERT的模型，BERT本身有两个输入
model = Model([x1_in, x2_in, img_input], p)

W1031 16:21:17.584550 140611263473472 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.



In [16]:
#LSTM版本
x = keras.layers.concatenate([img_output, text_output])

# 堆叠多个全连接网络层
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)

# 最后添加主要的逻辑回归层
output = Dense(3, activation='softmax', name='output')(x)

#定义一个具有两个输入和一个输出的模型：
model = Model(inputs=[img_input, text_input], outputs=[output])

In [10]:
from keras.optimizers import Adam

model.compile(
    optimizer=Adam(1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy'],
             )

W1031 16:21:54.668922 140611263473472 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [16]:
#文本数据处理
import codecs
import pandas as pd

token_dict = {}

with codecs.open(dict_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)


class OurTokenizer(Tokenizer):
    def _tokenize(self, text):
        R = []
        for c in text:
            if c in self._token_dict:
                R.append(c)
            elif self._is_space(c):
                R.append('[unused1]') # space类用未经训练的[unused1]表示
            else:
                R.append('[UNK]') # 剩余的字符是[UNK]
        return R

tokenizer = OurTokenizer(token_dict)

neg = pd.read_excel('/openbayes/input/input1/neg.xls', header=None)
pos = pd.read_excel('/openbayes/input/input1/pos.xls', header=None)
neu = pd.read_excel('/openbayes/input/input1/neu.xls', header=None)

data = []

for d in neg[0]:
    data.append((d, 0))

for d in pos[0]:
    data.append((d, 1))
    
for d in neu[0]:
    data.append((d, 2))


# 按照9:1的比例划分训练集和验证集
random_order = list(range(len(data)))
np.random.shuffle(random_order)
train_data = [data[j] for i, j in enumerate(random_order) if i % 10 != 0]
valid_data = [data[j] for i, j in enumerate(random_order) if i % 10 == 0]


def seq_padding(X, padding=0):
    L = [len(x) for x in X]
    ML = max(L)
    return np.array([
        np.concatenate([x, [padding] * (ML - len(x))]) if len(x) < ML else x for x in X
    ])


class data_generator:
    def __init__(self, data, batch_size=32):
        self.data = data
        self.batch_size = batch_size
        self.steps = len(self.data) // self.batch_size
        if len(self.data) % self.batch_size != 0:
            self.steps += 1
    def __len__(self):
        return self.steps
    def __iter__(self):
        while True:
            idxs = list(range(len(self.data)))
            np.random.shuffle(idxs)
            X1, X2, Y = [], [], []
            for i in idxs:
                d = self.data[i]
                text = d[0][:maxlen]
                x1, x2 = tokenizer.encode(first=text)
                y = d[1]
                X1.append(x1)
                X2.append(x2)
                Y.append([y])
                if len(X1) == self.batch_size or i == idxs[-1]:
                    X1 = seq_padding(X1)
                    X2 = seq_padding(X2)
                    Y = seq_padding(Y)
                    yield [X1, X2], Y
                    [X1, X2, Y] = [], [], []

                    
train_D = data_generator(train_data)
valid_D = data_generator(valid_data)